# **The Battle of Neighborhoods - Part 2**


New York City's demographics show that it is a large and ethnically diverse metropolis. With it's diverse culture , comes diverse food items. There are many resturants in New york City, each beloning to different categories like Chinese , Italian , French etc.

So as part of this project , we will list and visualize all major parts of New York City that has great French resturants.

## Data

For this project we need the following data:

1.  New York City data that contains list Boroughs, Neighborhoods along with their latitude and longitude.
2.  French resturants in each neighborhood of New York city.
3.  GeoSpace data

## Questions that will be answered at the end of this project

1.  What are best location in New York City for French Cuisine?
2.  Which areas have potential French Restaurant Market?
3.  Which areas lack French Restaurants?
4.  Which is the best place to stay if you prefer French Cuisine?

## Methodology

1.  We begin by collecting the New York city data from "https://cocl.us/new_york_dataset"
2.  We will find all venues for each neighborhood using FourSquare API
3.  We will then filter out all French Restuarant venues
4.  Next using FourSquare API, we will find the Ratings, Tips, and Like count for all the French Resturants
5.  Next we will sort the data keeping ratings as the constraint
6.  Finally, we will visualize the Ranking of neighborhoods using python's Folium library



### Install Modules

In [3]:
!pip install geocoder
!pip install folium
!pip install BeautifulSoup4

### Import Libraries

In [4]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup
import geocoder
import os
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import plotly.graph_objects as go
%matplotlib inline

print('Done')

Done



The follwoing function get the geocodes i.e latitude and longitude of a given location using geopy.

In [5]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude

The following function interacts with FourSquare API and gets top 100 venues within a radius of 1000 metres for a given latitude and longitude. Below function returns the venue id , venue name and category.

In [7]:
def get_venues(lat,lng):
    
    #set variables
    radius=1000
    LIMIT=100
    CLIENT_ID = 'XHUUZX3T2H3I14E2B3QXON3VGMVZ52IXL2NBG021H5NIV432' # your Foursquare ID
    CLIENT_SECRET = 'FUY424BMLDUOQMQGLUYGIME3UMV4TPEPITDR2YNQS0A2YF1R' # your Foursquare Secret
    VERSION = '20200722' # Foursquare API version
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

The following function counts likes, tips for a given venue id.

In [8]:
def get_venue_details(venue_id):
        
    CLIENT_ID = 'XHUUZX3T2H3I14E2B3QXON3VGMVZ52IXL2NBG021H5NIV432' # your Foursquare ID
    CLIENT_SECRET = 'FUY424BMLDUOQMQGLUYGIME3UMV4TPEPITDR2YNQS0A2YF1R' # your Foursquare Secret
    VERSION = '20200722' # Foursquare API version
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id = venue_data['id']
        venue_name = venue_data['name']
        venue_likes = venue_data['likes']['count']
        venue_rating = venue_data['rating']
        venue_tips = venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


The following funtion gets the New York city data such as Boroughs, Neighborhoods along with their latitude and longitude.

In [9]:
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return new_york_data

Let's take a look at New York city data.

In [10]:
new_york_data= get_new_york_data()
new_york_data.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [11]:
new_york_data.shape

(306, 4)

The results above show that there are 306 different Neighborhoods in New York city.

In [12]:
ny_borough = new_york_data.groupby('Borough')['Neighborhood'].count()
ny_borough_df = pd.DataFrame(ny_borough)

In [13]:
boroughs = ny_borough_df.reset_index().Borough
count_neighborhoods = ny_borough_df.reset_index().Neighborhood
fig = go.Figure([go.Bar(x = boroughs, y = count_neighborhoods, name ='Neighborhood')])
fig.update_layout(title = 'Number of Neighborhoods for each Borough in New York City', title_x=0.5, plot_bgcolor='rgb(230, 230,230)', showlegend=True, font=dict( family="Calibri", size=18, color="black"))
fig.update_xaxes(title_text='Boroughs', nticks=20)
fig.update_yaxes(title_text='Number of Neighborhoods')
fig.show()


From the above bar plot, we can see that Queens has the highest number of neighborhoods.

In [12]:
# prepare neighborhood list that contains French resturants
column_names=['Borough', 'Neighborhood', 'ID','Name']
french_rest_ny=pd.DataFrame(columns=column_names)
count=1
for row in new_york_data.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    french_resturants=venues[venues['Category']=='French Restaurant']   
    print('(',count,'/',len(new_york_data),')','French Resturants in '+Neighborhood+', '+Borough+':'+str(len(french_resturants)))
    for resturant_detail in french_resturants.values.tolist():
        id, name , category=resturant_detail
        french_rest_ny = french_rest_ny.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1

( 1 / 306 ) French Resturants in Wakefield, Bronx:0
( 2 / 306 ) French Resturants in Co-op City, Bronx:0
( 3 / 306 ) French Resturants in Eastchester, Bronx:0
( 4 / 306 ) French Resturants in Fieldston, Bronx:0
( 5 / 306 ) French Resturants in Riverdale, Bronx:0
( 6 / 306 ) French Resturants in Kingsbridge, Bronx:0
( 7 / 306 ) French Resturants in Marble Hill, Manhattan:0
( 8 / 306 ) French Resturants in Woodlawn, Bronx:0
( 9 / 306 ) French Resturants in Norwood, Bronx:0
( 10 / 306 ) French Resturants in Williamsbridge, Bronx:0
( 11 / 306 ) French Resturants in Baychester, Bronx:0
( 12 / 306 ) French Resturants in Pelham Parkway, Bronx:0
( 13 / 306 ) French Resturants in City Island, Bronx:1
( 14 / 306 ) French Resturants in Bedford Park, Bronx:0
( 15 / 306 ) French Resturants in University Heights, Bronx:0
( 16 / 306 ) French Resturants in Morris Heights, Bronx:0
( 17 / 306 ) French Resturants in Fordham, Bronx:0
( 18 / 306 ) French Resturants in East Tremont, Bronx:0
( 19 / 306 ) Fre

In [13]:
french_rest_ny.to_csv('french_rest_ny.csv',index=False)

In [14]:
french_rest_ny = pd.read_csv('french_rest_ny.csv')

In [15]:
french_rest_ny.head()

,Borough,Neighborhood,ID,Name
0,Bronx,City Island,4d7aa83ee540f04d66ba16dc,Bistro SK
1,Brooklyn,Greenpoint,5bf358689fca56002c251a87,Citroën
2,Brooklyn,Greenpoint,58fd34584382ab70ad90083e,Chez Ma Tante
3,Brooklyn,Greenpoint,4bcf1883c564ef3ba33beff0,Le Gamin
4,Brooklyn,Windsor Terrace,4e7d174cb6340a4da75a2cca,Le Paddock


In [16]:
french_rest_ny.shape

(82, 4)

From the results above, we see that there are 82 French resturants across New York City

In [17]:
ny_french_res_df = pd.DataFrame(french_rest_ny.groupby('Borough')['Name'].count())
ny_french_res_df.head()

,Name
Borough,
Bronx,1
Brooklyn,26
Manhattan,47
Queens,6
Staten Island,2


In [18]:
borough_rest = ny_french_res_df.reset_index().Borough
count_rest = ny_french_res_df.reset_index().Name
fig = go.Figure([go.Bar(x = borough_rest, y = count_rest, name ='Resturants')])
fig.update_layout(title = 'Number of French Restaurants for each Borough in New York City', title_x=0.5, plot_bgcolor='rgb(230, 230,230)', showlegend=True, font=dict( family="Calibri", size=18, color="black"))
fig.update_xaxes(title_text='Borough', nticks=20)
fig.update_yaxes(title_text='Number of French Restaurants')
fig.show()

From the above Bar Plot, we can see that Manhattan has the highest number of French resturants.

In [19]:
ny_french_res_nh_df = pd.DataFrame(french_rest_ny.groupby('Neighborhood')['Name'].count().nlargest(5))
ny_french_res_nh_df.reset_index()

,Neighborhood,Name
0,Downtown,4
1,Lincoln Square,4
2,Central Harlem,3
3,Greenpoint,3
4,Lenox Hill,3


In [20]:
neighborhood_rest = ny_french_res_nh_df.reset_index().Neighborhood
count_rest_nh = ny_french_res_nh_df.reset_index().Name
fig = go.Figure([go.Bar(x = neighborhood_rest, y = count_rest_nh, name ='Resturants')])
fig.update_layout(title = 'Number of French Restaurants for each Neighborhood in New York City', title_x=0.5, plot_bgcolor='rgb(230, 230,230)', showlegend=True, font=dict( family="Calibri", size=18, color="black"))
fig.update_xaxes(title_text='Neighborhood', nticks=20)
fig.update_yaxes(title_text='Number of French Restaurants')
fig.show()

Based on the above results, Downtown and Lincoln Square have the highest number of French Resturants with a total count of 4.

In [21]:
french_rest_manh = french_rest_ny[french_rest_ny['Borough']=='Manhattan'].reset_index().drop(columns='index')
french_rest_manh.head()

,Borough,Neighborhood,ID,Name
0,Manhattan,Chinatown,57583641498e90001a32e13e,Le Coucou
1,Manhattan,Chinatown,558ddcbe498e892dbe0d3a8b,Les Enfants de Bohème
2,Manhattan,Manhattanville,511a937ee88968b8acbba54b,Maison Harlem
3,Manhattan,Central Harlem,4b80389ff964a520e95d30e3,Yatenga
4,Manhattan,Central Harlem,59cc6158c0af57432b9fd02b,Renaissance Harlem


In [22]:
french_rest_manh.shape

(47, 4)

Now we will get the ranking of each resturant for further analysis.

In [23]:
# prepare neighborhood list that contains French resturants
column_names=['Borough', 'Neighborhood', 'ID','Name','Likes','Rating','Tips']
french_rest_stats_manh = pd.DataFrame(columns=column_names)
count=1

for row in french_rest_manh.values.tolist():
    Borough,Neighborhood,ID,Name=row
    try:
        venue_details = get_venue_details(ID)
        print(venue_details)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except IndexError:
        print('No data available for id=',ID)
        # we will assign 0 value for these resturants as they may have been 
        #recently opened or details does not exist in FourSquare Database
        id,name,likes,rating,tips=[0]*5
    print('(',count,'/',len(french_rest_manh),')','processed')
    french_rest_stats_manh = french_rest_stats_manh.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)
    count+=1

                         ID       Name  Likes  Rating  Tips
0  57583641498e90001a32e13e  Le Coucou    394     9.3    90
( 1 / 47 ) processed
                         ID                   Name  Likes  Rating  Tips
0  558ddcbe498e892dbe0d3a8b  Les Enfants de Bohème     88     8.5    19
( 2 / 47 ) processed
                         ID           Name  Likes  Rating  Tips
0  511a937ee88968b8acbba54b  Maison Harlem    291     8.6    88
( 3 / 47 ) processed
                         ID     Name  Likes  Rating  Tips
0  4b80389ff964a520e95d30e3  Yatenga    104     8.7    62
( 4 / 47 ) processed
                         ID                Name  Likes  Rating  Tips
0  59cc6158c0af57432b9fd02b  Renaissance Harlem     14     8.0     2
( 5 / 47 ) processed
                         ID           Name  Likes  Rating  Tips
0  511a937ee88968b8acbba54b  Maison Harlem    291     8.6    88
( 6 / 47 ) processed
                         ID           Name  Likes  Rating  Tips
0  553c1b00498e7eb81316bf1b  Mountai

In [24]:
french_rest_stats_manh.shape

(47, 7)

In [25]:
french_rest_manh.shape


(47, 4)

In [26]:
french_rest_stats_manh.to_csv('french_rest_stats_manh.csv', index=False)

In [27]:
french_rest_stats_manh.head()

,Borough,Neighborhood,ID,Name,Likes,Rating,Tips
0,Manhattan,Chinatown,57583641498e90001a32e13e,Le Coucou,394,9.3,90
1,Manhattan,Chinatown,558ddcbe498e892dbe0d3a8b,Les Enfants de Bohème,88,8.5,19
2,Manhattan,Manhattanville,511a937ee88968b8acbba54b,Maison Harlem,291,8.6,88
3,Manhattan,Central Harlem,4b80389ff964a520e95d30e3,Yatenga,104,8.7,62
4,Manhattan,Central Harlem,59cc6158c0af57432b9fd02b,Renaissance Harlem,14,8.0,2


In [28]:
french_rest_stats_manh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Borough       47 non-null     object 
 1   Neighborhood  47 non-null     object 
 2   ID            47 non-null     object 
 3   Name          47 non-null     object 
 4   Likes         47 non-null     object 
 5   Rating        47 non-null     float64
 6   Tips          47 non-null     object 
dtypes: float64(1), object(6)
memory usage: 2.7+ KB


In [29]:
french_rest_stats_manh.Likes = french_rest_stats_manh.Likes.astype('float64')
french_rest_stats_manh.Tips = french_rest_stats_manh.Tips.astype('float64')

In [30]:
french_rest_stats_manh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Borough       47 non-null     object 
 1   Neighborhood  47 non-null     object 
 2   ID            47 non-null     object 
 3   Name          47 non-null     object 
 4   Likes         47 non-null     float64
 5   Rating        47 non-null     float64
 6   Tips          47 non-null     float64
dtypes: float64(3), object(4)
memory usage: 2.7+ KB



Lets find the French restuarants with Maximum Likes, Maximum Ratings and Maximum Tips.

In [31]:
# Resturant with maximum Likes
french_rest_stats_manh.iloc[french_rest_stats_manh['Likes'].idxmax()]

Borough                        Manhattan
Neighborhood                West Village
ID              4d0bf7e3f29c236ac675bfe7
Name                             Buvette
Likes                               1849
Rating                               9.2
Tips                                 521
Name: 30, dtype: object

In [32]:
# Resturant with maximum Rating
french_rest_stats_manh.iloc[french_rest_stats_manh['Rating'].idxmax()]

Borough                        Manhattan
Neighborhood                   Chinatown
ID              57583641498e90001a32e13e
Name                           Le Coucou
Likes                                394
Rating                               9.3
Tips                                  90
Name: 0, dtype: object

In [33]:
# Resturant with maximum Tips
french_rest_stats_manh.iloc[french_rest_stats_manh['Tips'].idxmax()]

Borough                        Manhattan
Neighborhood                West Village
ID              4d0bf7e3f29c236ac675bfe7
Name                             Buvette
Likes                               1849
Rating                               9.2
Tips                                 521
Name: 30, dtype: object

Lets visualize neighborhood with maximum average rating of resturants.

In [34]:
ny_neighborhood_stats = french_rest_stats_manh.groupby('Neighborhood',as_index=False).mean()[['Neighborhood','Rating']]
ny_neighborhood_stats.columns=['Neighborhood','Average Rating']
ny_neighborhood_stats.sort_values(['Average Rating'],ascending=False).head(10)

,Neighborhood,Average Rating
13,Little Italy,9.300000
26,West Village,9.200000
7,East Harlem,9.100000
3,Chelsea,9.000000
20,Soho,9.000000
5,Civic Center,8.950000
18,Noho,8.933333
14,Lower East Side,8.900000
4,Chinatown,8.900000
9,Greenwich Village,8.850000


Above are the top neighborhoods with top average rating of French resturants.

Same way lets find the average number of likes for French Resturants for each Borough.

In [35]:
ny_neighborhood_like_stats = french_rest_stats_manh.groupby('Neighborhood',as_index=False).mean()[['Neighborhood','Likes']]
ny_neighborhood_like_stats.columns=['Neighborhood','Average num. of Likes']
ny_neighborhood_like_stats.sort_values(['Average num. of Likes'],ascending=False).head(10)

,Neighborhood,Average num. of Likes
26,West Village,1849.000000
22,Tribeca,718.000000
5,Civic Center,556.000000
27,Yorkville,521.000000
13,Little Italy,394.000000
12,Lincoln Square,376.500000
20,Soho,337.333333
9,Greenwich Village,309.000000
16,Manhattanville,291.000000
1,Carnegie Hill,283.000000


In [56]:
ny_neighborhood_stats = french_rest_stats_manh.groupby('Neighborhood',as_index=False).mean()[['Neighborhood','Rating']]
ny_neighborhood_stats

,Neighborhood,Rating
0,Battery Park City,8.600000
1,Carnegie Hill,8.600000
2,Central Harlem,8.433333
3,Chelsea,9.000000
4,Chinatown,8.900000
5,Civic Center,8.950000
6,Clinton,8.200000
7,East Harlem,9.100000
8,Financial District,8.600000
9,Greenwich Village,8.850000


In [57]:
neighborhood_manh = ny_neighborhood_stats.Neighborhood
count_rest_rating = ny_neighborhood_stats.Rating
fig = go.Figure([go.Bar(x = neighborhood_manh, y = count_rest_rating, name ='Rating')])
fig.update_layout(title = 'Average rating of French Restaurants for each Manhattan neighborhood', title_x=0.5, plot_bgcolor='rgb(230, 230,230)', showlegend=True, font=dict( family="Calibri", size=18, color="black"))
fig.update_xaxes(title_text='Neighborhood', nticks=30)
fig.update_yaxes(title_text='Average rating of French Restaurants', nticks=15)
fig.show()


We will consider all the neighborhoods with average rating greater or equal 8.8 to visualize on map

In [58]:
ny_neighborhood_stats=ny_neighborhood_stats[ny_neighborhood_stats['Rating']>=8.8]
ny_neighborhood_stats


,Neighborhood,Rating
3,Chelsea,9.000000
4,Chinatown,8.900000
5,Civic Center,8.950000
7,East Harlem,9.100000
9,Greenwich Village,8.850000
13,Little Italy,9.300000
14,Lower East Side,8.900000
18,Noho,8.933333
20,Soho,9.000000
26,West Village,9.200000


In [59]:
ny_neighborhood_stats_merge = pd.merge(ny_neighborhood_stats,new_york_data, on='Neighborhood')

,Neighborhood,Rating,Borough,Latitude,Longitude
0,Chelsea,9.000000,Manhattan,40.744035,-74.003116
1,Chelsea,9.000000,Staten Island,40.594726,-74.189560
2,Chinatown,8.900000,Manhattan,40.715618,-73.994279
3,Civic Center,8.950000,Manhattan,40.715229,-74.005415
4,East Harlem,9.100000,Manhattan,40.792249,-73.944182
5,Greenwich Village,8.850000,Manhattan,40.726933,-73.999914
6,Little Italy,9.300000,Manhattan,40.719324,-73.997305
7,Lower East Side,8.900000,Manhattan,40.717807,-73.980890
8,Noho,8.933333,Manhattan,40.723259,-73.988434
9,Soho,9.000000,Manhattan,40.722184,-74.000657


In [62]:
ny_neighborhood_stats_merge = ny_neighborhood_stats_merge[ny_neighborhood_stats_merge['Borough'] == 'Manhattan']
ny_neighborhood_stats_merge

,Neighborhood,Rating,Borough,Latitude,Longitude
0,Chelsea,9.000000,Manhattan,40.744035,-74.003116
2,Chinatown,8.900000,Manhattan,40.715618,-73.994279
3,Civic Center,8.950000,Manhattan,40.715229,-74.005415
4,East Harlem,9.100000,Manhattan,40.792249,-73.944182
5,Greenwich Village,8.850000,Manhattan,40.726933,-73.999914
6,Little Italy,9.300000,Manhattan,40.719324,-73.997305
7,Lower East Side,8.900000,Manhattan,40.717807,-73.980890
8,Noho,8.933333,Manhattan,40.723259,-73.988434
9,Soho,9.000000,Manhattan,40.722184,-74.000657
10,West Village,9.200000,Manhattan,40.734434,-74.006180


Let's visualize it on New York map

In [71]:
ny_map = folium.Map(location=geo_location('New York'), zoom_start=12)

In [68]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the neighborhood and add each to the feature group
for lat, lng, in ny_neighborhood_stats_merge[['Latitude','Longitude']].values:
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10, # define how big you want the circle markers to be
            color='red',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

In [69]:
ny_neighborhood_stats_merge['Label']=ny_neighborhood_stats_merge['Neighborhood']+', '+ny_neighborhood_stats_merge['Borough']+'('+ny_neighborhood_stats_merge['Rating'].map(str)+')'

In [70]:
# add pop-up text to each marker on the map
for lat, lng, label in ny_neighborhood_stats_merge[['Latitude','Longitude','Label']].values:
    folium.Marker([lat, lng], popup=label).add_to(ny_map)        
# add incidents to map
ny_map.add_child(incidents)

Based on the above research we can answer the asked questions.

1.  Manhatten is the best location in New York City for French Cuisine
2.  Brooklyn has potential French Resturant Market.
3.  Bronx ranks last in average rating of French Resturants.
4.  Manhattan is the best place to stay if you love French Cuisine